In [1]:
import pandas as pd
from pulp import *

# Optimization using PuLP method
Based on Beer distribution example

## 1. Load Data

In [2]:
dfShip = pd.read_csv('ship_data.csv')
dfSupp = pd.read_csv('supply_data.csv')

## 2. Format Data

In [3]:
# Create dictionary w/ route tuple as key and cost as value
route_cost = dict(zip(zip(dfShip['node_orig'],dfShip['node_dest']), dfShip['cost']))

In [4]:
route_cost

{('PHX', 'ORD'): 6,
 ('PHX', 'ATL'): 7,
 ('PHX', 'DFW'): 3,
 ('PHX', 'LAX'): 3,
 ('AUS', 'LAX'): 7,
 ('AUS', 'DFW'): 2,
 ('AUS', 'ATL'): 5,
 ('GNV', 'DFW'): 6,
 ('GNV', 'ATL'): 4,
 ('GNV', 'JFK'): 7,
 ('DFW', 'LAX'): 5,
 ('DFW', 'ORD'): 4,
 ('DFW', 'JFK'): 6,
 ('DFW', 'ATL'): 2,
 ('ATL', 'JFK'): 5,
 ('ATL', 'ORD'): 4,
 ('ATL', 'DFW'): 2}

In [4]:
# Creat dictionary of each node and their respective supply or demand
orig_supply = dict(zip(dfSupp['node'],abs(dfSupp['supply'])))

In [13]:
# List +ve supply nodes as suppliers and -ve supply nodes as demanders
suppliers = list(dfSupp[dfSupp['supply'] > 0].node)
demanders = list(dfSupp[dfSupp['supply'] < 0].node)

## 3. Instantiate Problem and Vars using PuLP

In [8]:
prob = LpProblem("ShippingProblem",LpMinimize)

In [9]:
# Create route variable with a minimum 0 units and max 200 units 
route_vars = LpVariable.dicts("Route",(dfShip['node_orig'], dfShip['node_dest']),0,200,LpInteger)

## 4. Define shipping costs and set minimum constraints
all supplier nodes should give their supply and all demander nodes must meet their demand

In [11]:
prob += lpSum([route_vars[o][d]*route_cost[(o,d)] for (o,d) in route_cost.keys()]), "Sum of Transporting Costs"

In [14]:
# The supply maximum constraints are added to prob for each supply node 
for s in suppliers:
    prob += lpSum([route_vars[s][d] for d in demanders]) <= orig_supply[s], "Sum of Products out of Origin %s"%s

# The demand minimum constraints are added to prob for each demand node 
for d in demanders:
    prob += lpSum([route_vars[s][d] for s in suppliers]) >= orig_supply[d], "Sum of Products into Destination %s"%d

## 5. Write the problem to a file and solve

In [15]:
# The problem data is written to an .lp file
prob.writeLP("ShippingProblem.lp")

[Route_ATL_DFW,
 Route_ATL_JFK,
 Route_ATL_ORD,
 Route_AUS_ATL,
 Route_AUS_DFW,
 Route_AUS_JFK,
 Route_AUS_LAX,
 Route_AUS_ORD,
 Route_DFW_ATL,
 Route_DFW_JFK,
 Route_DFW_LAX,
 Route_DFW_ORD,
 Route_GNV_ATL,
 Route_GNV_DFW,
 Route_GNV_JFK,
 Route_GNV_LAX,
 Route_GNV_ORD,
 Route_PHX_ATL,
 Route_PHX_DFW,
 Route_PHX_JFK,
 Route_PHX_LAX,
 Route_PHX_ORD]

In [24]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_ATL_DFW = 0.0
Route_ATL_JFK = 0.0
Route_ATL_ORD = 0.0
Route_AUS_ATL = 0.0
Route_AUS_DFW = 100.0
Route_AUS_JFK = 50.0
Route_AUS_LAX = 0.0
Route_AUS_ORD = 50.0
Route_DFW_ATL = 0.0
Route_DFW_JFK = 0.0
Route_DFW_LAX = 0.0
Route_DFW_ORD = 0.0
Route_GNV_ATL = 0.0
Route_GNV_DFW = 0.0
Route_GNV_JFK = 0.0
Route_GNV_LAX = 50.0
Route_GNV_ORD = 150.0
Route_PHX_ATL = 150.0
Route_PHX_DFW = 200.0
Route_PHX_JFK = 200.0
Route_PHX_LAX = 150.0
Route_PHX_ORD = 0.0
Total Cost of Transportation =  2300.0


The solution yields a minimum, if I had more time, I would validate this solution